In [1]:
!pip install hyperopt

### HyperOpt 라이브러리의 주요 함수와 개념

- fmin: 이 함수는 최적화 과정을 수행하는 주요 함수입니다. 이 함수는 목표 함수(fn), 검색 공간(space), 최적화 알고리즘(algo), 그리고 최대 평가 횟수(max_evals) 등을 인자로 받습니다. 이 함수는 최적의 하이퍼파라미터 설정을 찾기 위해 주어진 알고리즘을 사용하여 검색 공간을 탐색하고, 각 설정에 대해 목표 함수를 평가합니다.

- tpe: 이것은 Tree-structured Parzen Estimator(TPE) 알고리즘을 나타내며, HyperOpt에서 사용할 수 있는 최적화 알고리즘 중 하나입니다. TPE는 베이지안 최적화 알고리즘의 한 종류로, 이전의 평가 결과를 기반으로 하여 다음에 테스트할 하이퍼파라미터를 선택합니다. 이렇게 하면 더 좋은 결과를 내는 하이퍼파라미터의 영역을 더 자주 탐색하게 됩니다.

- hp: 이것은 HyperOpt의 검색 공간을 정의하는 데 사용되는 함수들을 담고 있는 모듈입니다. 예를 들어, hp.uniform('x', 0, 1)은 0과 1 사이에서 균일하게 분포한 실수 값을 가지는 'x'라는 하이퍼파라미터를 정의합니다. 다른 함수로는 hp.choice (여러 선택 사항 중 하나를 선택), hp.normal (정규 분포에서 값을 선택) 등이 있습니다.

- STATUS_OK: 이것은 목표 함수가 성공적으로 완료되었음을 나타내는 상태 코드입니다. 목표 함수는 손실값과 상태 코드를 포함하는 딕셔너리를 반환해야 합니다. 이 상태 코드는 HyperOpt가 평가를 제대로 수행했는지 판단하는 데 사용됩니다. 만약 다른 값을 반환하면, HyperOpt는 해당 평가가 실패했다고 판단하고, 해당 설정을 더 이상 탐색하지 않습니다.

### Tree-structured Parzen Estimator (TPE)
- 하이퍼파라미터 최적화를 위한 알고리즘 중 하나입니다. Bayesian optimization의 한 종류로, 하이퍼파라미터의 새로운 조합을 선택하는 방법으로 주로 사용됩니다.

- TPE 알고리즘의 핵심 아이디어는 과거의 평가 결과를 기반으로 하이퍼파라미터의 좋은 값과 나쁜 값에 대한 확률 모델을 구축하고, 이 모델을 사용하여 다음에 시도할 하이퍼파라미터 값을 선택하는 것입니다.

- TPE는 이를 위해 두 개의 확률 분포 l(x)와 g(x)를 사용합니다. 여기서 x는 하이퍼파라미터를 의미하며, l(x)는 과거의 좋은 하이퍼파라미터 값들에 대한 분포를 나타내고, g(x)는 모든 과거의 하이퍼파라미터 값들에 대한 분포를 나타냅니다.

- 이 두 분포를 이용해 새로운 하이퍼파라미터 값 x를 선택할 때, l(x)/g(x)가 큰 값, 즉 과거의 좋은 하이퍼파라미터 값들에 더 가까운 값을 선택합니다.

- 이런 방식으로, TPE는 점점 더 좋은 하이퍼파라미터 값을 찾아가는 탐색 과정을 수행합니다. 이는 전통적인 Grid Search나 Random Search에 비해 효율적이며, 보다 적은 평가로 좋은 하이퍼파라미터 값을 찾을 수 있게 합니다.

### make_classification 함수 매개변수

- n_samples: 생성할 샘플의 개수를 지정합니다. 기본값은 100입니다.
- n_features: 독립 변수의 개수를 지정합니다. 기본값은 20입니다.
- n_informative: 종속 변수와 관련된 독립 변수의 수를 지정합니다. 기본값은 2입니다.
- n_redundant: 다른 독립 변수의 선형 조합으로 생성된 독립 변수의 수를 지정합니다. 기본값은 2입니다.
- n_classes: 생성할 클래스(또는 레이블)의 개수를 지정합니다. 기본값은 2입니다.

이 함수는 X와 y의 두 개의 배열을 반환합니다. X는 n_samples x n_features 크기의 2차원 배열로, 독립 변수를 포함하고 있습니다. y는 n_samples 크기의 1차원 배열로, 각 샘플의 클래스 레이블을 포함하고 있습니다.

In [2]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples = 1000, n_features = 10, n_classes = 2)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

X, y = make_classification()

def objective(args) :
    # 모델 설정 - c는 규제의 역수로 c 값이 작을수록 규제가 강하며 반대인 경우 규제가 약해집니다
    model = LogisticRegression(C = args['C'])
    # 모델 검증 - 교차검증을 수행하고 평균 정확도 계산
    score = cross_val_score(model, X, y, cv = 5).mean()
    # 손실값 계산 - 손실값을 1 - 정확도로 계산
    loss = 1 - score
    
    return{'loss' : loss, 'status' : STATUS_OK}

# 검색 공간 정의 - 최적화할 하이퍼파라미터 c의 범위를 정의
space = {'C' : hp.loguniform('C', -5, 0)}

#  최적화 실행 - tpe 알고리즘을 사용하여 목표 함수를 최적화하고  최대 100회의 평가를 수행합니다
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100)

print(best)

100%|█████████████████████████████████████████████| 100/100 [00:01<00:00, 50.70trial/s, best loss: 0.16999999999999993]
{'C': 0.7471525613377162}


### Hyperopt 사용법

- 입력 변수명과 입력값의 검색 공간 설정
- 목적 함수의 설정
- 목적 함수의 반환 최소값을 가지는 최적 입력값을 유추

In [4]:
search_space = {'x' : hp.quniform('x', -10, 10, 1), 'y' : hp.quniform('y', -15, 15, 1)}

In [5]:
def objective_func(search_space) :
    x = search_space['x']
    y = search_space['y']
    
    retval = x ** 2 - 20 * y
    
    return retval

In [6]:
from hyperopt import Trials
import numpy as np

trial_val = Trials()

best_01 = fmin(fn = objective_func, space = search_space, algo = tpe.suggest, max_evals = 5,
              trials = trial_val, rstate = np.random.default_rng(seed = 0))

print('best : ', best_01, '\n')

best_02 = fmin(fn = objective_func, space = search_space, algo = tpe.suggest, max_evals = 20,
              trials = trial_val, rstate = np.random.default_rng(seed = 0))

print('best : ', best_02)

100%|█████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 833.99trial/s, best loss: -224.0]
best :  {'x': -4.0, 'y': 12.0} 

100%|███████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 832.43trial/s, best loss: -296.0]
best :  {'x': 2.0, 'y': 15.0}


In [7]:
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}]


In [8]:
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, -6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0]}


In [9]:
import pandas as pd

losses = [loss_dict['loss'] for loss_dict in trial_val.results]

result_df = pd.DataFrame({'x' : trial_val.vals['x'], 'y' : trial_val.vals['y'], 'losses' : losses})

result_df.head()

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0


### 과제1 3개의 와인 클래스 분류

In [10]:
# Scikit-learn을 활용한 나이브 베이즈 분류기 구축 

from sklearn import datasets

#Load dataset
wine = datasets.load_wine()

# print the names of the 13 features
print("Features: ", wine.feature_names)

# print the label type of wine(class_0, class_1, class_2)
print("Labels: ", wine.target_names)

Features:  ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
Labels:  ['class_0' 'class_1' 'class_2']


In [27]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

gauss = GaussianNB()

gauss.fit(X_train, y_train)
pred = gauss.predict(X_test)

accuracy = accuracy_score(y_test, pred)

print('accuracy : ', accuracy)
print('class report : \n', metrics.classification_report(y_test, pred))

accuracy :  0.8888888888888888
class report : 
               precision    recall  f1-score   support

           0       0.91      1.00      0.95        10
           1       1.00      0.78      0.88        18
           2       0.73      1.00      0.84         8

    accuracy                           0.89        36
   macro avg       0.88      0.93      0.89        36
weighted avg       0.91      0.89      0.89        36



###  과제2  위스콘신 유방암 데이터 세트를 로딩해서 xgbooster 알고리즘으로 모델링 및 평가를 수행하세요 단 hyperopt 를 이용해서 하이퍼 파라미터를 최적화하세요

sklearn의 load_breast_cancer 함수를 사용하여 위스콘신 유방암 데이터 세트를 로딩합니다. 데이터를 훈련 데이터와 테스트 데이터로 분할한 후, Hyperopt의 fmin 함수를 사용하여 Objective 함수를 최적화합니다. Objective 함수는 XGBoost 모델을 초기화하고 훈련 데이터에 대해 학습한 뒤, 테스트 데이터에 대한 정확도를 계산하여 반환합니다. Hyperopt는 최적화를 위해 값을 최소화하므로, 음수로 정확도를 반환하도록 설정합니다. 최적화된 하이퍼파라미터는 best 변수에 저장되며, 최종 모델은 이를 사용하여 다시 학습하고 평가합니다.

위 코드를 실행하면 Hyperopt를 사용하여 XGBoost 모델의 하이퍼파라미터를 최적화하고, 최종적으로 테스트 데이터에 대한 정확도를 출력합니다.

In [17]:
import xgboost as xgb
from xgboost import plot_importance
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, Trials
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# 데이터 세트 로드
dataset = load_breast_cancer()

X = dataset.data
y = dataset.target

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

# 모델링을 위한 함수 정의
def objective(args) :
    
    # 하이퍼파라미터 추출
    learning_rate = args['learning_rate']
    max_depth = args['max_depth']
    subsample = args['subsample']
    
    # xgboost 모델 정의
    model = xgb.XGBClassifier(
        learning_rate = learning_rate,
        max_depth = max_depth,
        subsample = subsample,
        random_state = 42
    )
    
    # 모델 학습
    model.fit(X_train, y_train)
    
    # 검증 데이터에 대한 예측 수행
    y_pred = model.predict(X_test)
    
    # 정확도 계산
    accuracy = accuracy_score(y_test, y_pred)
    
    return -accuracy # Hyperopt는 최적화를 위해 값을 최소화하므로 음수로 반환

# 탐색 공간 정의
space = {
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth' : hp.choice('max_depth', range(1, 10)),
    'subsample' : hp.uniform('subsample', 0.5, 1.0)
}

# hyperopt를 사용하여 하이퍼 파라미터 최적화 및 출력
trials = Trials()
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100, trials = trials)

print('best hyperparameters : ', best)

# 최적화된 하이퍼 파라미터로 최종 모델 학습
best_model = xgb.XGBClassifier(
    learning_rate = best['learning_rate'],
    max_depth = best['max_depth'],
    subsample = best['subsample'],
    random_state = 10
)
best_model.fit(X_train, y_train)

# 최종 모델의 성능 평가
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print('final accuracy : ', accuracy)

100%|█████████████████████████████████████████████| 100/100 [00:07<00:00, 13.85trial/s, best loss: -0.9824561403508771]
best hyperparameters :  {'learning_rate': 0.06048296421915791, 'max_depth': 4, 'subsample': 0.6366215152608334}
final accuracy :  0.9736842105263158


In [22]:
from hyperopt import hp
from xgboost import XGBClassifier


dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data = dataset.data, columns = dataset.feature_names)
cancer_df['target'] = dataset.target

X = cancer_df.iloc[:, : -1]
y = cancer_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 10)


xgb_search_space = {
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    'max_depth' : hp.quniform('max_depth', 5, 20, 1),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 2, 1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1)
}

# 목적 함수 설정
def objective_func(search_space) :
    xgb_clf = XGBClassifier(n_estimators = 100, max_depth = int(search_space['max_depth']),
                            min_child_weight = int(search_space['min_child_weight']),
                            learning_rate = search_space['learning_rate'],
                            colsample_bytree = search_space['colsample_bytree'],
                            eval_metric = 'logloss'
                           )
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring = 'accuracy', cv = 3)
    
    return {'loss' : -1 * np.mean(accuracy), 'status' : STATUS_OK}

# 하이퍼 파라미터 도출
trial_val = Trials()
best = fmin(fn = objective_func,
           space = xgb_search_space,
           algo = tpe.suggest,
           max_evals = 50,
           trials = trial_val, rstate = np.random.default_rng(seed = 9))

print('best : ', best, '\n')
print('colsample_bytree : {0}, learning_rate : {1}, max_depth : {2}, min_child_weight : {3} \n'
     .format(round(best['colsample_bytree'], 5), round(best['learning_rate'], 5), int(best['max_depth']),
            int(best['min_child_weight'])))

# 성능평가
xgb_wrapper = XGBClassifier(n_estimators = 400,
    learning_rate = round(best['learning_rate'], 5),
    max_depth = int(best['max_depth']),
    min_child_weight = int(best['min_child_weight']),
    colsample_bytree = round(best['colsample_bytree'], 5)
    )

evals = [(X_tr, y_tr), (X_val, y_val)]

xgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds = 50, eval_metric = 'logloss',
               eval_set = evals, verbose = True)

prds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, preds, pred_proba)

100%|███████████████████████████████████████████████| 50/50 [00:12<00:00,  4.02trial/s, best loss: -0.9648106192633903]
best :  {'colsample_bytree': 0.6128583597981321, 'learning_rate': 0.10075744671102672, 'max_depth': 5.0, 'min_child_weight': 1.0} 

colsample_bytree : 0.61286, learning_rate : 0.10076, max_depth : 5, min_child_weight : 1 

[0]	validation_0-logloss:0.61052	validation_1-logloss:0.60971
[1]	validation_0-logloss:0.54034	validation_1-logloss:0.54231
[2]	validation_0-logloss:0.48226	validation_1-logloss:0.48072
[3]	validation_0-logloss:0.43265	validation_1-logloss:0.42732
[4]	validation_0-logloss:0.38891	validation_1-logloss:0.37954
[5]	validation_0-logloss:0.35286	validation_1-logloss:0.34103
[6]	validation_0-logloss:0.31989	validation_1-logloss:0.30673
[7]	validation_0-logloss:0.29231	validation_1-logloss:0.27715
[8]	validation_0-logloss:0.26650	validation_1-logloss:0.25203
[9]	validation_0-logloss:0.24354	validation_1-logloss:0.22907
[10]	validation_0-logloss:0.22322	val

[125]	validation_0-logloss:0.00963	validation_1-logloss:0.01541
[126]	validation_0-logloss:0.00957	validation_1-logloss:0.01546
[127]	validation_0-logloss:0.00950	validation_1-logloss:0.01516
[128]	validation_0-logloss:0.00945	validation_1-logloss:0.01494
[129]	validation_0-logloss:0.00938	validation_1-logloss:0.01509
[130]	validation_0-logloss:0.00933	validation_1-logloss:0.01515
[131]	validation_0-logloss:0.00926	validation_1-logloss:0.01504
[132]	validation_0-logloss:0.00922	validation_1-logloss:0.01524
[133]	validation_0-logloss:0.00914	validation_1-logloss:0.01509
[134]	validation_0-logloss:0.00909	validation_1-logloss:0.01512
[135]	validation_0-logloss:0.00907	validation_1-logloss:0.01495
[136]	validation_0-logloss:0.00901	validation_1-logloss:0.01500
[137]	validation_0-logloss:0.00895	validation_1-logloss:0.01489
[138]	validation_0-logloss:0.00889	validation_1-logloss:0.01514
[139]	validation_0-logloss:0.00887	validation_1-logloss:0.01500
[140]	validation_0-logloss:0.00883	valid

NameError: name 'get_clf_eval' is not defined